# Reinforcment Learning Project (Traing Flappybird)


## Defining the libarys and importing all libarys


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import pygame
from pygame.locals import *


## Defining game and images


In [ ]:
# Define some constants
SW = 280  # Screen width
SH = 511  # Screen height

BASEY = SH * 0.8
IMAGES = {}
pygame.font.init()  # Initialize Pygame
WINDOW = pygame.display.set_mode((SW, SH))
Font = pygame.font.SysFont("comicsans", 30)  # Load images and font
BIRD = "bird1.png"
BG = "bg.png"
PIPE = "pipe.png"
base = "base.png"
Q = np.zeros((7, 21, 2), dtype=float)
FPS = 32


this part is written with the help of Chat-GPT

## Functions for the game


In [ ]:
# The main game loop for static_func version
def static_func():
    bird_pos_x = int(SW/5)
    bird_pos_y = int((SH - IMAGES["bird"].get_height())/2)
    base_pos_x = 0
    while (True):
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()

            elif event.type == KEYDOWN and (event.key == K_SPACE or event.key == K_UP):
                return
            else:
                # Rendering the game elements on the screen
                WINDOW.blit(IMAGES["background"], (0, 0))
                WINDOW.blit(IMAGES["bird"], (bird_pos_x, bird_pos_y))
                WINDOW.blit(IMAGES["base"], (base_pos_x, BASEY))
                text1 = Font.render("AI PROJECT", 1, (255, 255, 255))
                text2 = Font.render("HARKISHAN SINGH", 1, (255, 255, 255))
                WINDOW.blit(text1, (SW/2, SH/2))
                WINDOW.blit(text2, (10, 50))
                pygame.display.update()
                FPSCLOCK.tick(FPS)

# The main game loop for playing the game


def start_game(generation, x, y):
    score = 0
    bird_pos_x = int(SW/5)
    bird_pos_y = int(SH/2)
    base_pos_x1 = 0
    base_pos_x2 = SW

    backg_x = 0
    backg_y = IMAGES["background"].get_width()

    newPipe1 = new_pipe()
    newPipe2 = new_pipe()

    up_pipes = [
        {"x": SW + 200, "y": newPipe1[0]["y"]},
        {"x": SW + 500, "y": newPipe2[0]["y"]}
    ]

    bttm_pipes = [
        {"x": SW+200, "y": newPipe1[1]["y"]},
        {"x": SW + 500, "y": newPipe2[1]["y"]}
    ]

    pipeVelx = -4

    birdvel = -9
    birdmaxvel = 10
    birdvelmin = -8
    birdyacc = 1

    player_Flap_Accv = -8
    player_Flapped = False

    while (True):

        x_prev, y_prev = convert(bird_pos_x, bird_pos_y, bttm_pipes)
        jump = rf_play(x_prev, y_prev)

        for event in pygame.event.get():
            if event.type == QUIT:
                plt.scatter(x, y)
                plt.xlabel("GENERATION/Number of Trials")
                plt.ylabel("SCORE")
                plt.title("Flappy Birds : AI Project")
                plt.show()
                pygame.quit()
                sys.exit()

        if jump:
            if bird_pos_y > 0:
                birdvel = player_Flap_Accv
                player_Flapped = True

        playerMidPos = bird_pos_x + IMAGES["bird"].get_width()/2
        for pipe in up_pipes:
            pipeMidPos = pipe["x"] + IMAGES["pipe"][0].get_width()/2
            if pipeMidPos <= playerMidPos < pipeMidPos + 4:
                score += 1

        if birdvel < birdmaxvel and not player_Flapped:
            birdvel += birdyacc

        if player_Flapped:
            player_Flapped = False

        playerHeight = IMAGES["bird"].get_height()

        bird_pos_y = bird_pos_y + min(birdvel, BASEY - bird_pos_y - playerHeight)

        for upperPipe, lowerPipe in zip(up_pipes, bttm_pipes):
            upperPipe["x"] += pipeVelx
            lowerPipe["x"] += pipeVelx

        if (0 < up_pipes[0]["x"] < 5):
            newPipe = new_pipe()
            up_pipes.append(newPipe[0])
            bttm_pipes.append(newPipe[1])

        if (up_pipes[0]["x"] < -IMAGES["pipe"][0].get_width()):
            up_pipes.pop(0)
            bttm_pipes.pop(0)
        base_pos_x1 -= 4
        base_pos_x2 -= 4
        if (base_pos_x1 <= -IMAGES["base"].get_width()):
            base_pos_x1 = base_pos_x2
            base_pos_x2 = base_pos_x1 + IMAGES["base"].get_width()

        backg_x -= 2
        backg_y -= 2
        if (backg_x <= -IMAGES["background"].get_width()):
            backg_x = backg_y
            backg_y = backg_x + IMAGES["background"].get_width()
        crashTest = collision(bird_pos_x, bird_pos_y, up_pipes, bttm_pipes)
        x_new, y_new = convert(bird_pos_x, bird_pos_y, bttm_pipes)
        if crashTest:
            reward = -1000
            update_q(x_prev, y_prev, jump, reward, x_new, y_new)
            return score

        reward = 15

        update_q(x_prev, y_prev, jump, reward, x_new, y_new)

        WINDOW.blit(IMAGES["background"], (backg_x, 0))
        WINDOW.blit(IMAGES["background"], (backg_y, 0))
        for upperPipe, lowerPipe in zip(up_pipes, bttm_pipes):
            WINDOW.blit(IMAGES["pipe"][0], (upperPipe["x"], upperPipe["y"]))
            WINDOW.blit(IMAGES["pipe"][1], (lowerPipe["x"], lowerPipe["y"]))
        WINDOW.blit(IMAGES["base"], (base_pos_x1, BASEY))
        WINDOW.blit(IMAGES["base"], (base_pos_x2, BASEY))
        text1 = Font.render("Score: " + str(score), 1, (255, 255, 255))
        text2 = Font.render(
            "Generation: " + str(generation), 1, (255, 255, 255))
        WINDOW.blit(text1, (SW - 10 - text1.get_width(), 10))
        WINDOW.blit(text2, (0, 0))
        WINDOW.blit(IMAGES["bird"], (bird_pos_x, bird_pos_y))

        pygame.display.update()
        FPSCLOCK.tick()

# Function to check collision with pipes or ground


def collision(bird_pos_x, bird_pos_y, up_pipes, bttm_pipes):
    if (bird_pos_y >= BASEY - IMAGES["bird"].get_height() or bird_pos_y < 0):
        return True
    for pipe in up_pipes:
        pipeHeight = IMAGES["pipe"][0].get_height()
        if (bird_pos_y < pipeHeight + pipe["y"] and abs(bird_pos_x - pipe["x"]) < IMAGES["pipe"][0].get_width()):
            return True

    for pipe in bttm_pipes:
        if (bird_pos_y + IMAGES["bird"].get_height() > pipe["y"] and abs(bird_pos_x - pipe["x"]) < IMAGES["pipe"][0].get_width()):
            return True
    return False


# Function to get new pipes
def new_pipe():

    pipeHeight = IMAGES["pipe"][1].get_height()
    gap = int(SH/4)
    y2 = int(gap + random.randrange(0,
             int(SH - IMAGES["base"].get_height() - 1.2*gap)))
    pipex = int(SW+300)
    y1 = int(pipeHeight - y2 + gap)

    pipe = [
        {"x": pipex, "y": -y1},
        {"x": pipex, "y": y2}
    ]
    return pipe
# Code from https://github.com/hsahib2912/AI-Flappy-Birds

## Deining the RF-Algorithm


In [ ]:
# AI function to control the bird's movement
def rf_play(x, y):
    max = 0
    jump = False

    if (Q[x][y][1] > Q[x][y][0]):
        max = Q[x][y][1]
        jump = True

    return jump


def converter(bird_pos_x, bird_pos_y, bttm_pipes):
    x = min(280, bttm_pipes[0]["x"])
    y = bttm_pipes[0]["y"]-bird_pos_y
    if (y < 0):
        y = abs(y)+408
    return int(x/40-1), int(y/40)


def update_q(x_prev, y_prev, jump, reward, x_new, y_new):

    if jump:
        Q[x_prev][y_prev][1] = 0.4 * Q[x_prev][y_prev][1] + \
            (0.6)*(reward+max(Q[x_new][y_new][0], Q[x_new][y_new][1]))
    else:
        Q[x_prev][y_prev][0] = 0.4 * Q[x_prev][y_prev][0] + \
            (0.6)*(reward+max(Q[x_new][y_new][0], Q[x_new][y_new][1]))

## Execute Main function


In [ ]:
if __name__ == "__main__":

    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    pygame.display.set_caption("RF_Learning_Project")

    IMAGES["base"] = pygame.image.load("base.png").convert_alpha()
    IMAGES["pipe"] = (pygame.transform.rotate(pygame.image.load(
        PIPE).convert_alpha(), 180), pygame.image.load(PIPE).convert_alpha())
    IMAGES["background"] = pygame.image.load(BG).convert()
    IMAGES["bird"] = pygame.image.load(BIRD).convert_alpha()
    generation = 1
    static_func()
    x = []
    y = []
    while (True):
        score = start_game(generation, x, y)
        if (score == -1):
            break
        x.append(generation)
        y.append(score)
        generation += 1

    print(generation)